In [ ]:
import os
import numpy as np
import pandas as pd
from PIL import Image
from torch.utils.data import Dataset, DataLoader
import torch
import torch.nn as nn
import torchvision.models as models
import torchvision.transforms as transforms
from sklearn.metrics import f1_score, accuracy_score
from torch.optim.lr_scheduler import StepLR
from sklearn.model_selection import train_test_split
from tqdm import tqdm

import torchvision.transforms as T
from torchvision.transforms import InterpolationMode

import pytorch_lightning
from monai.transforms import (
    Activations,
)

from monai.data import Dataset, DataLoader
from pathlib import Path
import torch
import numpy as np
from pytorch_lightning.callbacks import ModelCheckpoint
from torch.nn import BCEWithLogitsLoss
from torchmetrics import F1Score
from torch.optim.lr_scheduler import SequentialLR, LambdaLR, StepLR, SequentialLR
import ssl

from random import shuffle
import os
import random

from torchmetrics.classification import BinaryAUROC

import glob
import cv2
from skimage.filters import threshold_otsu
from scipy.stats import kurtosis, skew

import scipy
import scipy.ndimage as ndi

In [ ]:
class Net(pytorch_lightning.LightningModule):
    def __init__(self, model, optimizer, scheduler, train_loader, val_loader):
        super().__init__()
        self._model = model
        self._optimizer = optimizer
        self._scheduler = scheduler
        self.train_loader = train_loader
        self.val_loader = val_loader
        self.loss_function = BCEWithLogitsLoss()
        self.metric = F1Score(task='binary')
        self.activation = Activations(sigmoid=True)

    def forward(self, x):
        return self._model(x)

    def train_dataloader(self):
        return self.train_loader

    def val_dataloader(self):
        return self.val_loader

    def configure_optimizers(self):
        return {'optimizer': optimizer, 'lr_scheduler': scheduler}
    

    def training_step(self, batch, batch_idx):
        images, labels = batch
        labels = labels[:, None]
        output = self.forward(images)
        loss = self.loss_function(output, labels.float())
        self.log_dict({"training_loss": loss})
        return {"loss": loss}

    def validation_step(self, batch, batch_idx):
        images, labels = batch
        labels = labels[:, None]
        output = self.forward(images)
        loss = self.loss_function(output, labels.float())
        metric = self.metric(self.activation(output), labels.float())
        self.log_dict({"f1":metric, "loss": loss})
        return {"loss": loss}

In [ ]:
class ImageDataset(Dataset):
    def __init__(self, file_list, labels=None, transform=None):
        self.file_list = file_list
        self.labels = labels
        self.transform = transform

    def __len__(self):
        return len(self.file_list)

    def __getitem__(self, idx):
        img_path = self.file_list[idx]
        img = Image.open(img_path).convert("RGB")
        if self.transform:
            img = self.transform(img)
        if self.labels is not None:
            label = self.labels[idx]
            return img, label
        else:
            return img

In [ ]:
train_transforms = T.Compose([
    T.Resize(224, interpolation=InterpolationMode.BICUBIC),
    T.RandomResizedCrop(224),
    T.RandomHorizontalFlip(),
    T.RandomVerticalFlip(),
    T.RandomRotation(20),
    T.GaussianBlur(kernel_size=(7, 13), sigma=(0.1, 1.0)),
    T.ColorJitter(brightness=0.2, contrast=0.2, saturation=0.2, hue=0.1),
    T.ToTensor(),
    T.Normalize(mean=[0.485, 0.456, 0.406],
                std=[0.229, 0.224, 0.225]),
])

val_transforms = T.Compose([
    T.Resize(224, interpolation=InterpolationMode.BICUBIC),
    T.CenterCrop(224),
    T.ToTensor(),
    T.Normalize(mean=[0.485, 0.456, 0.406],
                std=[0.229, 0.224, 0.225]),
])

In [ ]:
def get_train_valid_split():
    '''
    Creates train/valid split for cnn. 
    Return train_paths, valid_paths, train_labels, valid_labels.
    '''
    
    base_dir = '/kaggle/input/ai-vs-human-generated-dataset'
    train_csv = '/kaggle/input/ai-vs-human-generated-dataset/train.csv'
    
    
    df_train = pd.read_csv(train_csv)
    
    
    df_train = df_train[['file_name', 'label']]
    
    df_train['file_name'] = df_train['file_name'].apply(lambda x: os.path.join(base_dir, x))
    all_image_paths = df_train['file_name'].values
    all_labels = df_train['label'].values
    train_paths, val_paths, train_labels, val_labels = train_test_split(all_image_paths, 
                                                                        all_labels, test_size=0.05,        
                                                                        random_state=43,
                                                                        shuffle=False)
    return train_paths, val_paths, train_labels, val_labels

In [ ]:
def create_dataloaders(train_paths, val_paths, train_labels, val_labels):
    """
    Returns train and valid dataloader.
    """
    batch_size = 32
    train_data = ImageDataset(train_paths, train_labels, transform=train_transforms)
    val_data   = ImageDataset(val_paths,   val_labels,   transform=val_transforms)
    train_loader = DataLoader(dataset=train_data, batch_size=batch_size, shuffle=True,  num_workers=4)
    val_loader   = DataLoader(dataset=val_data,   batch_size=batch_size, shuffle=False, num_workers=4)
    return train_loader, val_loader

In [ ]:
def create_model_optimizer_scheduler():
    """
    Returns model, optimizer and scheduler
    """

    model = models.convnext_base(weights="DEFAULT")
    
    for param in model.features.parameters():
        param.requires_grad = False
    
    for param in model.features[-2:].parameters(): 
        param.requires_grad = True
    
    model.classifier = nn.Sequential(
        nn.AdaptiveAvgPool2d((1, 1)),  
        nn.Flatten(),                  
        nn.BatchNorm1d(1024),          
        nn.Linear(1024, 512),          
        nn.ReLU(),                     
        nn.Dropout(0.4),               
        nn.Linear(512, 1)              
    )
    
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    model = model.to(device)
    
    optimizer = torch.optim.AdamW([
        {'params': model.features[-2:].parameters(), 'lr': 1e-5},
        {'params': model.classifier.parameters(), 'lr': 1e-4}     
    ])
    
    scheduler = StepLR(optimizer, step_size=5, gamma=0.7)

    return model, optimizer, scheduler

In [ ]:
def train_model(model, optimizer, scheduler, train_loader, val_loader):
    """
    Trains model
    """
    net = Net(model, optimizer, scheduler, train_loader, val_loader)
    trainer = pytorch_lightning.Trainer(
            devices=[0],
            max_epochs=5,
            enable_checkpointing=True,
            num_sanity_val_steps=1,
            log_every_n_steps=16,
            callbacks=[ModelCheckpoint('models/', '{f1:.2f}_{epoch}', monitor='f1', mode='max')]
        )
    trainer.fit(net)
    return net

In [ ]:
train_paths, val_paths, train_labels, val_labels = get_train_valid_split()

train_loader, val_loader = create_dataloaders(train_paths, val_paths, train_labels, val_labels)

model, optimizer, scheduler = create_model_optimizer_scheduler()

net = train_model(model, optimizer, scheduler, train_loader, val_loader)

In [ ]:
# Paths
MODEL_PATH = "/kaggle/working/best_model_mobilenetv3"
TEST_CSV_PATH = "/kaggle/input/ai-vs-human-generated-dataset/test.csv"
TEST_IMAGES_PATH = "/kaggle/input/ai-vs-human-generated-dataset/test_data_v2"
OUTPUT_CSV_PATH = "/kaggle/working/submission.csv"

# Load the model
model = tf.keras.models.load_model(MODEL_PATH)

# Load the test CSV
test_df = pd.read_csv(TEST_CSV_PATH)

# Image preprocessing function
def preprocess_image(img_path, target_size=(224, 224)):
    img = image.load_img(img_path, target_size=target_size)
    img_array = image.img_to_array(img)
    img_array = np.expand_dims(img_array, axis=0)  # Add batch dimension
    #img_array /= 255.0  # Normalize
    return img_array

# Make predictions
predictions = []
for img_path in test_df['id']:
    img_filename = os.path.basename(img_path)  # Extract only the filename
    full_img_path = os.path.join(TEST_IMAGES_PATH, img_filename)

    if not os.path.exists(full_img_path):  # Debugging check
        print(f"File not found: {full_img_path}")

    img_array = preprocess_image(full_img_path)
    pred = model.predict(img_array)[0][0]  # Get prediction score
    label = 0 if pred > 0.5 else 1
    predictions.append(label)

# Save predictions to CSV
submission_df = pd.DataFrame({'id': test_df['id'], 'label': predictions})